# Лабораторная работа 5. Вариант 3. Группа 343

## Задание
Найти решение задачи

$\frac{\partial u}{\partial t}=Lu+f(x,t),0<x<1,0<1\leq0.1,$

$u(x,0)=\varphi(x),0\leq x\leq1$

$(\alpha_1(t)u-\alpha_2(t)\frac{\partial u}{\partial x})|_{x=0}=\alpha(t), 0\leq t\leq0.1,$

$(\beta_1(t)u+\beta_2(t)\frac{\partial u}{\partial x})|_{x=1}=\beta(t), 0\leq t\leq0.1,$

используя различные разностные схемы:

+ явную схему порядка $O(h^2+\tau)$ с аппроксимацией производных в граничных условиях с порядком $O(h^2)$;
+ схему с весами при $\sigma=0,\sigma=1,\sigma=1/2$ с аппроксимацией производных в граничных условиях с порядком $O(h)$.

### Условие
$\frac{\partial u}{\partial t}=\frac{\partial^2 u}{\partial {x^2}}+\frac{\partial u}{\partial x}+f(x,t),$

$(a(x,t)=1, b(x,t)=1, c(x,t)=0)$

$u(x,0)=\varphi(x),0\leq x\leq 1,$

$u(0,t)=\alpha(t), \frac{\partial u}{\partial x}|_{x=1}=\beta(t),0\leq t\leq 0.1.$

$(\alpha_1=1, \alpha_2=0, \beta_1=0, \beta_2=1)$

### Функции для тестирования
$u_1(x, t) = x^3 + t^3$ (выберем эту)

$u_2(x, t) = x^3t^3$

$u_3(x, t) = cos(2x) + sin(2t + 1)$

$u_4(x, t) = cos(2x)sin(2t + 1)$

$u_5(x, t) = 2x - t$

In [1]:
import numpy as np
from numpy import linalg as LA
from tabulate import tabulate
import warnings

N = 5
M = 5
X_RANGE = (0, 1)
T_RANGE = (0, 0.1)

ALPHA_1 = 1
ALPHA_2 = 0
BETA_1 = 0
BETA_2 = 1

US_STRINGS = ['x^3 + t^3', 'x^3 * t^3', 'cos(2x) + sin(2t+1)', 'cos(2x)sin(2t+1)', '2x - t']

In [2]:
# (функция, первая, вторая производная по х, первая по t)
def u1(x, t):
    u = x ** 3 + t ** 3
    dxu = 3 * x ** 2
    d2xu = 6 * x
    dtu = 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u2(x, t):
    u = x ** 3 * t ** 3
    dxu = 3 * x ** 2 * t ** 3
    d2xu = 6 * x * t ** 3
    dtu = 3 * x ** 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u3(x, t):
    u = np.cos(2 * x) + np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x)
    d2xu = -4 * np.cos(2 * x)
    dtu = 2 * np.sin(2 * t + 1)
    return (u, dxu, d2xu, dtu)


def u4(x, t):
    u = np.cos(2 * x) * np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x) * np.sin(2 * t + 1)
    d2xu = -4 * np.cos(2 * x) * np.sin(2 * t + 1)
    dtu = 2 * np.cos(2 * t + 1) * np.cos(2 * x)
    return (u, dxu, d2xu, dtu)

def u5(x, t):
    u = 2 * x - t
    dxu = 2
    d2xu = 0
    dtu = -1
    return (u, dxu, d2xu, dtu)


US = [u1, u2, u3, u4, u5]

In [3]:
def a(x, t):
    return 1


def b(x, t):
    return 1


def c(x, t):
    return 0

In [4]:
def find_system_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
                
        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x

In [5]:
def find_tables_norm(table_a, table_b):
    res = -1
    for i in range(0, 5):
        for j in range(0, 5):
            a = np.abs(table_a[i, j])
            b = np.abs(table_b[i, j])
            res = np.max([res, np.abs(a - b)])
    return res

# f = du/dt - d2u/dx2 - du/dx
def find_accurate_solution(u, x, t):
    solution = u(x, t)[0]
    f_value = u(x, t)[3] - u(x, t)[2] - u(x, t)[1]
    return (solution, f_value)


def find_accurate_solutions(n, m, u):
    res = np.zeros((n + 1, m + 1))
    step_x = (X_RANGE[1] - X_RANGE[0]) / n
    step_t = (T_RANGE[1] - T_RANGE[0]) / m
    
    for i in range(0, n + 1):
        for k in range(0, m + 1):
            x = X_RANGE[0] + i * step_x
            t = T_RANGE[0] + k * step_t
            res[i, k] = u(x, t)[0]
            
    return res

$u_0^k(\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k))-\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}=\alpha(t_k)$

$u_0^k=\frac{\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}+\alpha(t_k)}{\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k)}$

$u_0^k=\frac{\alpha_2(t_k)(4u_1^k-u_2^k)+2h\alpha(t_k)}{2h\alpha_1(t_k)+3\alpha_2(t_k)}$


---


$u_N^k(\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k))-\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}=\beta(t_k)$

$u_N^k=\frac{\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}+\beta(t_k)}
{\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k)}$

$u_N^k=\frac{\beta_2(t_k)(4u_{N-1}^k - u_{N-2}^k)+2h\beta(t_k)}{2h\beta_1(t_k)+3\beta_2(t_k)}$


---


$L_hu_i^k=a(x_i, t_k)\frac{u_{i+1}^k-2u_i^k+u_{i-1}^k}{h^2}+b(x_i,t_k)\frac{u_{i+1}^k-u_{i-1}^k}{2h}+c(x_i, t_k)u_i^k$

In [6]:
def find_solution_grid(n, m, u, a, b, c):
    step_x = (X_RANGE[1] - X_RANGE[0]) / n
    step_t = (T_RANGE[1] - T_RANGE[0]) / m
    # us[i, k] <-> (u_i)^k
    us = np.zeros((n + 1, m + 1))
    # u: (x, t) -> (u, dxu, d2xu, dtu)
    def put_u_i_k(i, k):
        x_i = X_RANGE[0] + i * step_x
        # t_(k-1), t_k
        t_k1 = T_RANGE[0] + (k - 1) * step_t
        t_k = t_k1 + step_t
        
        # (u_i)^0
        if k == 0:
            us[i, k] = u(x_i, T_RANGE[0])[0]
            return
        
        # (u_0)^k
        if i == 0:
            alpha = ALPHA_1 * u(0, t_k)[0] + ALPHA_2 * u(0, t_k)[1]
            u0k =  ALPHA_2 * (4 * us[1, k] - us[2, k]) + 2 * step_x * alpha
            u0k /= 2 * step_x * ALPHA_1 + 3 * ALPHA_2
            us[i, k] = u0k
            return
        
        # (u_N)^k
        if i == n:
            beta = BETA_1 * u(1, t_k)[0] + BETA_2 * u(1, t_k)[1]
            uNk = BETA_2 * (4 * us[n - 1, k] - us[n - 2, k]) + 2 * step_x * beta
            uNk /= 2 * step_x * BETA_1 + 3 * BETA_2
            us[i, k] = uNk
            return
    
        # (u_i)^k-1
        u_prev = us[i, k - 1]
        
        # Lh
        lh = \
            a(x_i, t_k) * (us[i + 1, k] - 2 * us[i, k] + us[i - 1, k]) / step_x ** 2 + \
            0.5 * b(x_i, t_k) * (us[i + 1, k] - us[i - 1, k]) / step_x + \
            c(x_i, t_k) * us[i, k]
        # f(x_i, t_(k-1))
        function_val = find_accurate_solution(u, x_i, t_k1)[1]
        # (u_i)^k
        us[i, k] = u_prev + step_t * (lh * u_prev + function_val)
        
    for i in range(0, n + 1):
        put_u_i_k(i, 0)
        
    for k in range(1, m + 1):
        for i in range(1, n):
            put_u_i_k(i, k)
        put_u_i_k(0, k)
        put_u_i_k(n, k)
            
    return us

In [7]:
# AS: 0,A_1..A_n+1
# BS: B_0..B_n+1
# CS: C_0..C_n
# GS: G_0..G_n+1
# find y_0..y_n+1
def find_solution_tridiag(AS, BS, CS, GS):
    n = len(AS) - 1
    ss = np.zeros(n + 1)
    ts = np.zeros(n + 1)
    # Решение: набор y_i
    ys = np.zeros(n + 1)
    ss[0] = CS[0] / BS[0]
    ts[0] = - GS[0] / BS[0]
    
    # Ищем s_i, t_i
    for i in range(1, n + 1):
        denom = BS[i] - AS[i] * ss[i - 1]
        ss[i] = CS[i] / denom
        ts[i] = (AS[i] * ts[i - 1] - GS[i]) / denom
    
    # find y_i
    ys[n] = ts[n]
    # for n-1..0
    for i in range(n - 1, -1, -1):
        ys[i] = ss[i] * ys[i + 1] + ts[i]
    
    return ys

Имеем:

$\alpha_1(t_k)u_0^k-\alpha_2(t_k)\frac{u_1^k-u_0^k}{h}=\alpha(t_k)$

$\beta_1(t_k)u_N^k+\beta_2(t_k)\frac{u_N^k-u_{N-1}^k}{h}=\beta(t_k)$

Нужно:

$-B_0u_0^k+C_0u_1^k=G_0^k$

$A_Nu_{N-1}^k-B_Nu_N^k=G_N^k$


$(\alpha_1(t_k)+\frac{\alpha_2(t_k)}{h})u_0^k-\frac{\alpha_2(t_k)}{h}u_1^k=\alpha(t_k)$

$(\beta_1(t_k)+\frac{\beta_2(t_k)}{h})u_N^k-\frac{\alpha_2(t_k)}{h}u_{N-1}^k=\beta(t_k)$

Для удобства можно домножить с двух сторон на h

In [8]:
def find_solution_grid_weights(sigma, n, m, u, a, b, c):
    step_x = (X_RANGE[1] - X_RANGE[0]) / n
    step_t = (T_RANGE[1] - T_RANGE[0]) / m
    # us[i, k] <-> (u_i)^k
    us = np.zeros((n + 1, m + 1))
    gs = np.zeros((n + 1, m + 1))
    
    def put_g_i_k(i, k):
        x_i = X_RANGE[0] + i * step_x
        t_k = T_RANGE[0] + k * step_t
        
        # find Lh
        lh = \
            a(x_i, t_k) * (us[i + 1, k] - 2 * us[i, k] + us[i - 1, k]) / step_x ** 2 + \
            0.5 * b(x_i, t_k) * (us[i + 1, k] - us[i - 1, k]) / step_x + \
            c(x_i, t_k) * us[i, k]
        
        g = - 1 / step_t * us[i, k - 1]
        g -= (1 - sigma) * lh * us[i, k - 1]
        # f(x_i, t_k)
        function_val = find_accurate_solution(u, x_i, t_k)[1]
        g -= function_val
        gs[i, k] = g
        
    # (u_i)^0
    for i in range(0, n + 1):
        x_i = X_RANGE[0] + i * step_x
        us[i, 0] = u(x_i, T_RANGE[0])[0]
    
    AS = np.zeros((n + 1, m + 1))
    BS = np.zeros((n + 1, m + 1))
    CS = np.zeros((n + 1, m + 1))
    
    # (u_i)^k
    for k in range(1, m + 1):
        t_k = T_RANGE[0] + k * step_t
        x_i = X_RANGE[0] + k * step_x
        # Ишем Gs
        # (G_0)^k
        alpha = ALPHA_1 * u(0, t_k)[0] - ALPHA_2 * u(0, t_k)[1]
        gs[0, k] = alpha * step_x
        # (G_i)^k для i=1..N-1
        for i in range(1, n):
            put_g_i_k(i, k)
        # (G_N)^k
        beta = BETA_1 * u(1, t_k)[0] - BETA_2 * u(1, t_k)[1]
        gs[n, k] = beta * step_x
        
        # a, b, c
        AS[0, k] = 0
        BS[0, k] = - ALPHA_1 * step_x - ALPHA_2
        CS[0, k] = - ALPHA_2
        
        for i in range(1, n):
            sigma_a = sigma * a(x_i, t_k) / step_x ** 2
            sigma_b = b(x_i, t_k) / (2 * step_x)
            AS[i, k] = sigma_a - sigma_b
            BS[i, k] = 2 * sigma_a + 1 / step_t
            CS[i, k] = sigma_a + sigma_b
        
        AS[n, k] = - BETA_1 * step_x - BETA_2
        BS[n, k] = - BETA_2
        CS[n, k] = 0
        
        # (G_0)^k..(G_N)^k
        g_ks = gs[:, k]
        # Решаем систему, получаем (u_i)^k для текущего k
        u_is = find_solution_tridiag(AS[:, k], BS[:, k], CS[:, k], g_ks)
        us[:,k] = u_is
    
    return us

In [9]:
def print_result(u, is_explicit=True, sigma=0, not_explicit_m=10):
    if is_explicit:
        print('Явный результат:')
        us = find_solution_grid(N, M, u, a, b, c)
    else:
        print(f'Неявный результат для веса {sigma}:')
        us = find_solution_grid_weights(sigma, N, M, u, a, b, c)
        
    us_headers = ['t \ x']
    for i in range(0, N + 1):
        us_headers.append(str(round(X_RANGE[0] + i * (X_RANGE[1] - X_RANGE[0]) / N, 2)))
        
    us_table = []
    for k in range(0, M + 1):
        us_table.append([str(round(T_RANGE[0] + k * (T_RANGE[1] - T_RANGE[0]) / M, 2))])
        
    for i in range(0, N + 1):
        for k in range(0, M + 1):
            us_table[k].append(str(us[i, k]))            
        
    print(tabulate(us_table, headers=us_headers))
    
    print('Результат:')
    if is_explicit:
        res_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau1)||']
    else:
        res_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau)||']
        
    res_table = []
    # N = (x1 - x0) / h
    # M = (t1 - t0) / tau
    for (h, tau, tau1) in [(0.2, 0.01, 0.01), (0.1, 0.01, 0.01), (0.05, 0.01, 0.01), (0.025, 0.01, 0.01), (0.0125, 0.01, 0.01)]:
        n = int((X_RANGE[1] - X_RANGE[0]) / h)
        nx2 = int((X_RANGE[1] - X_RANGE[0]) / (2 * h))
        m = int((T_RANGE[1] - T_RANGE[0]) / tau)
        m1 = int((T_RANGE[1] - T_RANGE[0]) / tau1)
        row = []
        row.append(h)
        row.append(tau)
        acc_table = find_accurate_solutions(n, m, u)
        if is_explicit:
            # u^(h,tau)
            table_1 = find_solution_grid(n, m, u, a, b, c)
            # u^(2h,tau1)
            table_2 = find_solution_grid(nx2, m1, u, a, b, c)
        else:
            tau = 0.1 / not_explicit_m
            # u^(h,tau)
            table_1 = find_solution_grid_weights(sigma, n, m, u, a, b, c)
            # u^(2h,tau)
            table_2 = find_solution_grid_weights(sigma, nx2, m, u, a, b, c)
        
        row.append(find_tables_norm(acc_table, table_1))
        
        if len(res_table) != 0:
            row.append(find_tables_norm(table_1, table_2))
        
        res_table.append(row)
    
    print(tabulate(res_table, headers=res_headers))

In [10]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')   
    test_u = US[0]
    print(f'Результат для тестового u: {US_STRINGS[0]}\n')
    print_result(test_u, True)
    print()
    print_result(test_u, False, 1)

Результат для тестового u: x^3 + t^3

Явный результат:
  t \ x       0.0        0.2          0.4         0.6          0.8       1.0
-------  --------  ---------  -----------  ----------  -----------  --------
   0     0          0.008      0.064        0.216       0.512       1
   0.02  8e-06     -0.0184     0.00587008   0.122971    0.405932    0.900253
   0.04  6.4e-05   -0.044776  -0.0518242    0.0265268   0.276402    0.759694
   0.06  0.000216  -0.07108   -0.107671    -0.0682625   0.133608    0.600898
   0.08  0.000512  -0.097264  -0.160342    -0.156721   -0.00999909  0.438908
   0.1   0.001     -0.12328   -0.208663    -0.235221   -0.142957    0.287798
Результат:
     h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau1)||
------  -----  -----------------------  -------------------------------
0.2      0.01               0.22673
0.1      0.01               0.0577249                         0.448
0.05     0.01               0.0311369                         0.056
0.025    